In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [10]:
fashion_mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
range_start = 20
plt.figure(figsize=(10,10))

for i in range(range_start,range_start + 10):
    plt.subplot(5,5,i-range_start+1)
    plt.xticks([])
    plt.xlabel(str(train_labels[i]))
    plt.imshow(train_images[i])
plt.show()

In [12]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [13]:
train_images[0].size

784

In [44]:
ds = tf.data.Dataset.from_tensors(train_images)

In [45]:
dsb = ds.batch(100)

In [46]:
dsb.element_spec

TensorSpec(shape=(None, 60000, 28, 28), dtype=tf.float64, name=None)

In [47]:
j=0
for i in dsb:
    j = j + 1
j

1

In [14]:
model=None
model = tf.keras.Sequential()

model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 100)               78500     
                                                                 
 dense_4 (Dense)             (None, 10)                1010      
                                                                 
 dense_5 (Dense)             (None, 10)                110       
                                                                 
Total params: 79620 (311.02 KB)
Trainable params: 79620 (311.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.03),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 100)               78500     
                                                                 
 dense_4 (Dense)             (None, 10)                1010      
                                                                 
 dense_5 (Dense)             (None, 10)                110       
                                                                 
Total params: 79620 (311.02 KB)
Trainable params: 79620 (311.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
history = model.fit(train_images, train_labels, epochs=60,verbose=1,batch_size=1,validation_split=0.3)


Epoch 1/60
42000/42000 [==============================] - 83s 2ms/step - loss: 0.3525 - accuracy: 0.9042 - val_loss: 0.2647 - val_accuracy: 0.9303
Epoch 2/60
42000/42000 [==============================] - 82s 2ms/step - loss: 0.2312 - accuracy: 0.9411 - val_loss: 0.2644 - val_accuracy: 0.9363
Epoch 3/60
22850/42000 [===============>..............] - ETA: 38s - loss: 0.1984 - accuracy: 0.9480

KeyboardInterrupt: 

In [ ]:
loss, acc = model.evaluate(train_images,  train_labels, verbose=2)
print("\nTraining accuracy: ", acc)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("\nTesting accuracy: ", acc)

In [ ]:
def plot_loss(history,data_names):
    n = 1
    plt.figure(n,figsize=(20,5))

    for i in data_names:
        plt.plot(history.history[i],label=i)
        plt.xlim([0, len(history.history[i])])
        plt.ylim([np.min(history.history[i]), np.max(history.history[i])])
        plt.xlabel('Epoch')
        #plt.ylabel(i)
        n += 1

    plt.legend()
    plt.grid(True)
    plt.show()

plot_loss(history,['loss','val_loss'])
plot_loss(history,['accuracy','val_accuracy'])



In [ ]:
#pred_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = model.predict(test_images)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(str(predicted_label),
                                         100*np.max(predictions_array),
                                         str(true_label)),
               color=color)


def plot_value_array(i, predictions_array, true_label):
    true_label = true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
range_start = 100
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(range_start,range_start+num_images):
    j = i - range_start
    plt.subplot(num_rows, 2*num_cols, 2*j+1)
    plot_image(i, predictions[i], test_labels, test_images)
    plt.subplot(num_rows, 2*num_cols, 2*j+2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()